In [4]:
# Import Library.
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# Device Configuration.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters.
num_epochs = 5
batch_size = 4
learning_rate = 0.001

"""
Dataset has PILImage images of range [0, 1].
Transform them to Tensors of normalized range [-1, 1].
"""
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class.
train_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)

classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)


def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)  # -> n, 400
        x = F.relu(self.fc1(x))  # -> n, 120
        x = F.relu(self.fc2(x))  # -> n, 84
        x = self.fc3(x)  # -> n, 10
        return x


model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

# Model Training Loop.
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        # Forward Pass and Loss Calculation.
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward Pass and Weight's Updation.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 2000 == 0:
            print(
                f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.3f}"
            )

print("Finished Training.")
PATH = "./cnn.pth"
torch.save(model.state_dict(), PATH)

# Testing Model Performance.
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f"The accuracy of the CNN Model is {acc} %")

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f"Accuracy of {classes[i]}: {acc} %")

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5], Step [2000/12500], Loss: 2.332
Epoch [1/5], Step [4000/12500], Loss: 2.259
Epoch [1/5], Step [6000/12500], Loss: 2.288
Epoch [1/5], Step [8000/12500], Loss: 2.285
Epoch [1/5], Step [10000/12500], Loss: 2.234
Epoch [1/5], Step [12000/12500], Loss: 2.055
Epoch [2/5], Step [2000/12500], Loss: 2.223
Epoch [2/5], Step [4000/12500], Loss: 1.832
Epoch [2/5], Step [6000/12500], Loss: 1.265
Epoch [2/5], Step [8000/12500], Loss: 1.263
Epoch [2/5], Step [10000/12500], Loss: 1.832
Epoch [2/5], Step [12000/12500], Loss: 1.131
Epoch [3/5], Step [2000/12500], Loss: 1.928
Epoch [3/5], Step [4000/12500], Loss: 0.942
Epoch [3/5], Step [6000/12500], Loss: 1.827
Epoch [3/5], Step [8000/12500], Loss: 2.803
Epoch [3/5], Step [10000/12500], Loss: 1.783
Epoch [3/5], Step [12000/12500], Loss: 2.343
Epoch [4/5], Step [2000/12500], Loss: 1.314
Epoch [4/5], Step [4000/12500], Loss: 1.430
Epoch [4/5], Step [6000/12500], Loss: 